In [ ]:
from bs4 import BeautifulSoup
import urllib, os, urllib.request
import re
import requests
from mutagen.mp3 import MP3
from mutagen.id3 import ID3, TIT2
import os
import sys
import csv


lyricsPath = os.getcwd()+'\\songs_lyrics'
if not os.path.exists(lyricsPath):
    os.makedirs(lyricsPath)
lyricsPath

In [ ]:
def urlify(s):

    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)

    # Replace all runs of whitespace with a single dash
    s = re.sub(r"\s+", '-', s)

    return s


In [ ]:
def generateUrls():
    print("----------- generating urls -----------")
    path = 'D:\MUSIC\sounds\RAP\Eminem'
    files = []
    urls = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.mp3' in file :
                files.append(os.path.join(r, file))
                
    for f in files:
        try :
            tags = ID3(os.path.join(r,f))
            if 'TIT2' in tags:
                #songName = urlify(tags['TIT2'].text[0])
                songName = tags['TIT2'].text[0]
                songName = re.sub(r"feat+.([^\w]|[\w])*","",songName,flags=re.IGNORECASE)
                songName = urlify(songName)
                songName = songName.replace("'",'')
                if songName[-1]=='-':
                    songName = songName[:-1]
                url = "https://genius.com/eminem-"+songName+"-lyrics"
                urls.append(url)
        except :
            tags = ID3()
    urls = list(dict.fromkeys(urls))
    urls
    urlsFile = open(os.path.expanduser(r"urls.csv"),"wb")
    for url in urls:
        urlsFile.write(bytes(url+"\n",encoding="ascii", errors='ignore'))
    urlsFile.close()

In [ ]:
def generateWorkingAndNotWorkingUrlsFile():
    print("----------- generating files -----------")
    urls = []
    notWorkingUrls = []
    workingURLS =[]
    with open(os.path.expanduser(r"urls.csv"),"r") as urlsFile:
       # urls = csv.reader(urlsFile, delimiter=' ', quotechar='|')
        for url in urlsFile:
            urls.append(url)
    urlsFile.close()

    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Referer': 'https://cssspritegenerator.com',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}
    for url in urls:
        try:
            req=urllib.request.Request(url,headers=hdr)
            thePage = urllib.request.urlopen(req)
            soup = BeautifulSoup(thePage, "html.parser")
            title = soup.find('div',class_='header_with_cover_art-primary_info').h1.text
            title = re.sub(r'/',"-",title)
            workingURLS.append(url)
        except urllib.error.HTTPError as err:
            if err.code == 404:
                notWorkingUrls.append(url)
            else:
                raise
    workingURLS = list(dict.fromkeys(workingURLS))
    notWorkingUrls = list(dict.fromkeys(notWorkingUrls))
    urls = list(dict.fromkeys(urls))
    notUrlsFile = open(os.path.expanduser(r"not-working-urls.csv"),"wb")
    for url in notWorkingUrls :
        urls.remove(url)
        notUrlsFile.write(bytes(url,encoding="ascii", errors='ignore'))
    notUrlsFile.close()
    urlsFile = open(os.path.expanduser(r"working-urls.csv"),"wb")
    for url in workingURLS:
        urlsFile.write(bytes(url,encoding="ascii", errors='ignore'))
    urlsFile.close()

In [ ]:
def getWorkingUrls():
    urls = []
    i=0
    with open(os.path.expanduser(r"working-urls.csv"),"r") as urlsFile:
        for url in urlsFile:
            urls.append(url)
    urlsFile.close()
    urls.sort()
    urls = list(dict.fromkeys(urls))
    return urls

In [ ]:
def getExistingUrls():
    existingUrls = []
    if os.path.isfile(lyricsPath+r'\existing-urls.csv'):
        with open(os.path.expanduser(lyricsPath+r'\existing-urls.csv'),'r') as existingUrlsFile:
            for url in existingUrlsFile:
                url = re.sub(r'.*url : ', '',url)
                existingUrls.append(url)
        existingUrlsFile.close()
        existingUrls.sort()
        return existingUrls

In [ ]:
def scrappingWorkingUrls():
    print("----------- scrapping urls -----------")
    pathExistingUrlsFile= lyricsPath+r'\existing-urls.csv'
    pathExistingUrlsFile
    if os.path.isfile(pathExistingUrlsFile):
        existingUrlsFile = open(os.path.expanduser(pathExistingUrlsFile),'ab')
    else:
        existingUrlsFile = open(os.path.expanduser(pathExistingUrlsFile),'wb')

    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
             'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
             'Referer': 'https://cssspritegenerator.com',
             'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
             'Accept-Encoding': 'none',
             'Accept-Language': 'en-US,en;q=0.8',
             'Connection': 'keep-alive'}
    urls = getWorkingUrls()
    existingUrls = getExistingUrls()
    for url in urls :
        if  not url in existingUrls:
            try:
                req=urllib.request.Request(url,headers=hdr)
                thePage = urllib.request.urlopen(req)
                soup = BeautifulSoup(thePage, "lxml")
                lyrics = soup.find('div', class_="lyrics").p.text
                title = soup.find('div',class_='header_with_cover_art-primary_info').h1.text
                title = re.sub(r'/',"-",title)
                title = re.sub(r'[^\w\s]|[\-\(\)\']','',title)
                artist = soup.find('div',class_='header_with_cover_art-primary_info').h2.text.lstrip().rstrip()
                if not os.path.exists(lyricsPath+"\\"+title+".txt"):
                    songFile = open(os.path.expanduser(lyricsPath+"\\"+title+".txt"),"wb")
                    songFile.write(bytes("Title : "+title+"\nArtist : "+artist+"\nLyrics :\n"+lyrics,encoding="ascii", errors='ignore'))
                    songFile.close()
                    existingUrlsFile.write(bytes("Title : "+title+" , url : "+url,encoding="ascii", errors='ignore'))
                    print(urls[urls.index(url)])
            except urllib.error.HTTPError as err:
                raise
    existingUrlsFile.close()
    print('Done')

In [ ]:
if not os.path.isfile(r'urls.csv'):
    generateUrls()
generateWorkingAndNotWorkingUrlsFile()
scrappingWorkingUrls()

In [18]:

from pymongo import MongoClient
from random import randint
#Step 1: Connect to MongoDB - Note: Change connection string as needed
client = MongoClient(port=27017)
db=client.business
#Step 2: Create sample data
names = ['Kitchen','Animal','State', 'Tastey', 'Big','City','Fish', 'Pizza','Goat', 'Salty','Sandwich','Lazy', 'Fun']
company_type = ['LLC','Inc','Company','Corporation']
company_cuisine = ['Pizza', 'Bar Food', 'Fast Food', 'Italian', 'Mexican', 'American', 'Sushi Bar', 'Vegetarian']
for x in range(1, 501):
    business = {
        'name' : names[randint(0, (len(names)-1))] + ' ' + names[randint(0, (len(names)-1))]  + ' ' + company_type[randint(0, (len(company_type)-1))],
        'rating' : randint(1, 5),
        'cuisine' : company_cuisine[randint(0, (len(company_cuisine)-1))] 
    }
    #Step 3: Insert business object directly into MongoDB via isnert_one
    result=db.reviews.insert_one(business)
    #Step 4: Print to the console the ObjectID of the new document
    print('Created {0} of 100 as {1}'.format(x,result.inserted_id))
#Step 5: Tell us that you are done
print('finished creating 100 business reviews')

Created 1 of 100 as 5d972d77462583ebb92e8a95
Created 2 of 100 as 5d972d77462583ebb92e8a96
Created 3 of 100 as 5d972d77462583ebb92e8a97
Created 4 of 100 as 5d972d77462583ebb92e8a98
Created 5 of 100 as 5d972d77462583ebb92e8a99
Created 6 of 100 as 5d972d77462583ebb92e8a9a
Created 7 of 100 as 5d972d77462583ebb92e8a9b
Created 8 of 100 as 5d972d77462583ebb92e8a9c
Created 9 of 100 as 5d972d77462583ebb92e8a9d
Created 10 of 100 as 5d972d77462583ebb92e8a9e
Created 11 of 100 as 5d972d77462583ebb92e8a9f
Created 12 of 100 as 5d972d77462583ebb92e8aa0
Created 13 of 100 as 5d972d77462583ebb92e8aa1
Created 14 of 100 as 5d972d77462583ebb92e8aa2
Created 15 of 100 as 5d972d77462583ebb92e8aa3
Created 16 of 100 as 5d972d77462583ebb92e8aa4
Created 17 of 100 as 5d972d77462583ebb92e8aa5
Created 18 of 100 as 5d972d77462583ebb92e8aa6
Created 19 of 100 as 5d972d77462583ebb92e8aa7
Created 20 of 100 as 5d972d77462583ebb92e8aa8
Created 21 of 100 as 5d972d77462583ebb92e8aa9
Created 22 of 100 as 5d972d77462583ebb92e8a

Created 304 of 100 as 5d972d77462583ebb92e8bc4
Created 305 of 100 as 5d972d77462583ebb92e8bc5
Created 306 of 100 as 5d972d77462583ebb92e8bc6
Created 307 of 100 as 5d972d77462583ebb92e8bc7
Created 308 of 100 as 5d972d77462583ebb92e8bc8
Created 309 of 100 as 5d972d77462583ebb92e8bc9
Created 310 of 100 as 5d972d77462583ebb92e8bca
Created 311 of 100 as 5d972d77462583ebb92e8bcb
Created 312 of 100 as 5d972d77462583ebb92e8bcc
Created 313 of 100 as 5d972d77462583ebb92e8bcd
Created 314 of 100 as 5d972d77462583ebb92e8bce
Created 315 of 100 as 5d972d77462583ebb92e8bcf
Created 316 of 100 as 5d972d77462583ebb92e8bd0
Created 317 of 100 as 5d972d77462583ebb92e8bd1
Created 318 of 100 as 5d972d77462583ebb92e8bd2
Created 319 of 100 as 5d972d77462583ebb92e8bd3
Created 320 of 100 as 5d972d77462583ebb92e8bd4
Created 321 of 100 as 5d972d77462583ebb92e8bd5
Created 322 of 100 as 5d972d77462583ebb92e8bd6
Created 323 of 100 as 5d972d77462583ebb92e8bd7
Created 324 of 100 as 5d972d77462583ebb92e8bd8
Created 325 o

In [19]:
from pymongo import MongoClient
from random import randint
#Step 1: Connect to MongoDB - Note: Change connection string as needed
client = MongoClient(port=27017)
print(client)
db=client.business

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
